In [456]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

In [457]:
df_train=pd.read_excel('doctortrain.xlsx')
df_test=pd.read_excel('doctortest.xlsx')

In [458]:
df_train.shape,df_test.shape

((5961, 7), (1987, 6))

In [459]:
df_train.isnull().sum()

Qualification            0
Experience               0
Rating                3302
Place                   25
Profile                  0
Miscellaneous_Info    2620
Fees                     0
dtype: int64

In [460]:
df_test.isnull().sum()

Qualification            0
Experience               0
Rating                1090
Place                    6
Profile                  0
Miscellaneous_Info     834
dtype: int64

In [461]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 7 columns):
Qualification         5961 non-null object
Experience            5961 non-null object
Rating                2659 non-null object
Place                 5936 non-null object
Profile               5961 non-null object
Miscellaneous_Info    3341 non-null object
Fees                  5961 non-null int64
dtypes: int64(1), object(6)
memory usage: 326.1+ KB


In [462]:
df_train.nunique()

Qualification         1420
Experience              64
Rating                  51
Place                  877
Profile                  6
Miscellaneous_Info    2998
Fees                    40
dtype: int64

In [463]:
df_test.isnull().sum()

Qualification            0
Experience               0
Rating                1090
Place                    6
Profile                  0
Miscellaneous_Info     834
dtype: int64

In [464]:
df_train.dtypes

Qualification         object
Experience            object
Rating                object
Place                 object
Profile               object
Miscellaneous_Info    object
Fees                   int64
dtype: object

In [465]:
df_test.dtypes

Qualification         object
Experience            object
Rating                object
Place                 object
Profile               object
Miscellaneous_Info    object
dtype: object

In [466]:
df_train

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250
...,...,...,...,...,...,...,...
5956,"MBBS, MS - ENT",19 years experience,98%,"Basavanagudi, Bangalore",ENT Specialist,"98% 45 Feedback Basavanagudi, Bangalore",300
5957,MBBS,33 years experience,NaN,"Nungambakkam, Chennai",General Medicine,NaN,100
5958,MBBS,41 years experience,97%,"Greater Kailash Part 2, Delhi",General Medicine,"97% 11 Feedback Greater Kailash Part 2, Delhi",600
5959,"MBBS, MD - General Medicine",15 years experience,90%,"Vileparle West, Mumbai",General Medicine,General Medical Consultation Viral Fever Treat...,100


In [467]:
df_train['Experience'] = df_train['Experience'].str.replace('years experience','').astype(int)
df_test['Experience'] = df_test['Experience'].str.replace('years experience','').astype(int)

In [468]:
df_train['Experience']

0       24
1       12
2        9
3       12
4       20
        ..
5956    19
5957    33
5958    41
5959    15
5960    17
Name: Experience, Length: 5961, dtype: int32

In [469]:
df_train['Qualification']

0                BHMS, MD - Homeopathy
1         BAMS, MD - Ayurveda Medicine
2       MBBS, MS - Otorhinolaryngology
3                  BSc - Zoology, BAMS
4                                 BAMS
                     ...              
5956                    MBBS, MS - ENT
5957                              MBBS
5958                              MBBS
5959       MBBS, MD - General Medicine
5960           BDS, MDS - Orthodontics
Name: Qualification, Length: 5961, dtype: object

In [470]:
length=list(df_train['Qualification'])
max=1
for i in length:
    if len(i.split(',')) > max:
        max=len(i.split(','))
print('the max length in a cell is ',max)

the max length in a cell is  10


In [471]:
import re

In [472]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df_train['Qualification'] = df_train['Qualification'].apply(clean_text)
df_test['Qualification'] = df_test['Qualification'].apply(clean_text)

In [473]:
df_train['Qualification']

0                bhms md homeopathy
1         bams md ayurveda medicine
2       mbbs ms otorhinolaryngology
3                  bsc zoology bams
4                              bams
                   ...             
5956                    mbbs ms ent
5957                           mbbs
5958                           mbbs
5959       mbbs md general medicine
5960           bds mds orthodontics
Name: Qualification, Length: 5961, dtype: object

In [474]:
def clean_place(text):
    text=str(text).lower()
    text=re.sub(r'[^a-z0-9]',' ',text)
    text=re.sub(r'\s+',' ',text)
    return text


df_train['Place'] = df_train['Place'].apply(clean_place)
df_test['Place'] = df_test['Place'].apply(clean_place)


In [475]:
df_train['Place']

0                 kakkanad ernakulam
1               whitefield bangalore
2            mathikere bel bangalore
3        bannerghatta road bangalore
4               keelkattalai chennai
                    ...             
5956          basavanagudi bangalore
5957            nungambakkam chennai
5958    greater kailash part 2 delhi
5959           vileparle west mumbai
5960                 pitampura delhi
Name: Place, Length: 5961, dtype: object

In [476]:
df_train['Place'].fillna('missing',inplace=True)
df_test['Place'].fillna('missing',inplace=True)

In [477]:
df_train['Rating'].fillna('0%',inplace=True)
df_test['Rating'].fillna('0%',inplace=True)

def clean_rating(text):
    text=str(text)
    text=re.sub(r'%','',text)
    return text

df_train['Rating']=df_train['Rating'].apply(clean_rating)
df_test['Rating']=df_test['Rating'].apply(clean_rating)

df_train['Rating']=df_train['Rating'].astype(int)
df_test['Rating']=df_test['Rating'].astype(int)


In [478]:
df_train['Miscellaneous_Info'].fillna('missing', inplace=True)
df_test['Miscellaneous_Info'].fillna('missing', inplace=True)

def clean_misc(text):
    text=str(text).lower()
    text=re.sub(r'[^a-z0-9]',' ',text)
    text=re.sub(r',',' ',text)
    text=re.sub(r'%',' ',text)
    text=re.sub(r'₹',' ',text)
    text=re.sub(r'\s+',' ',text)
    return text

df_train['Miscellaneous_Info']=df_train['Miscellaneous_Info'].apply(clean_misc)
df_test['Miscellaneous_Info']=df_test['Miscellaneous_Info'].apply(clean_misc)

In [479]:
df_test.isnull().sum(),df_train.isnull().sum()

(Qualification         0
 Experience            0
 Rating                0
 Place                 0
 Profile               0
 Miscellaneous_Info    0
 dtype: int64, Qualification         0
 Experience            0
 Rating                0
 Place                 0
 Profile               0
 Miscellaneous_Info    0
 Fees                  0
 dtype: int64)

In [480]:
df_train = pd.get_dummies(df_train, columns=['Profile'])
df_test  = pd.get_dummies(df_test,  columns=['Profile'])

In [481]:
df_train

,Qualification,Experience,Rating,Place,Miscellaneous_Info,Fees,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath
0,bhms md homeopathy,24,100,kakkanad ernakulam,100 16 feedback kakkanad ernakulam,100,0,0,0,0,0,1
1,bams md ayurveda medicine,12,98,whitefield bangalore,98 76 feedback whitefield bangalore,350,1,0,0,0,0,0
2,mbbs ms otorhinolaryngology,9,0,mathikere bel bangalore,missing,300,0,0,0,1,0,0
3,bsc zoology bams,12,0,bannerghatta road bangalore,bannerghatta road bangalore 250 available on s...,250,1,0,0,0,0,0
4,bams,20,100,keelkattalai chennai,100 4 feedback keelkattalai chennai,250,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5956,mbbs ms ent,19,98,basavanagudi bangalore,98 45 feedback basavanagudi bangalore,300,0,0,0,1,0,0
5957,mbbs,33,0,nungambakkam chennai,missing,100,0,0,0,0,1,0
5958,mbbs,41,97,greater kailash part 2 delhi,97 11 feedback greater kailash part 2 delhi,600,0,0,0,0,1,0
5959,mbbs md general medicine,15,90,vileparle west mumbai,general medical consultation viral fever treat...,100,0,0,0,0,1,0


In [482]:
df_train.dtypes ,df_test.dtypes

(Qualification               object
 Experience                   int32
 Rating                       int32
 Place                       object
 Miscellaneous_Info          object
 Fees                         int64
 Profile_Ayurveda             uint8
 Profile_Dentist              uint8
 Profile_Dermatologists       uint8
 Profile_ENT Specialist       uint8
 Profile_General Medicine     uint8
 Profile_Homeopath            uint8
 dtype: object, Qualification               object
 Experience                   int32
 Rating                       int32
 Place                       object
 Miscellaneous_Info          object
 Profile_Ayurveda             uint8
 Profile_Dentist              uint8
 Profile_Dermatologists       uint8
 Profile_ENT Specialist       uint8
 Profile_General Medicine     uint8
 Profile_Homeopath            uint8
 dtype: object)

In [483]:
df_qualification=pd.DataFrame(df_train['Qualification'].str.split(n=9).tolist(),columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10'])

In [484]:
df_qualification_test=pd.DataFrame(df_test['Qualification'].str.split(n=9).tolist(),columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10'])

In [485]:
for col in df_qualification.columns:
    df_qualification[col]=df_qualification[col].astype(str)

In [486]:
for col in df_qualification.columns:
    df_qualification_test[col]=df_qualification_test[col].astype(str)

In [487]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in df_qualification.columns:
    df_qualification[col]=le.fit_transform(df_qualification[col])

In [488]:
for col in df_qualification.columns:
    df_qualification_test[col]=le.fit_transform(df_qualification_test[col])

In [489]:
df_train['Place']=le.fit_transform(df_train['Place'])
df_test['Place']=le.fit_transform(df_test['Place'])

In [490]:
df_train.drop(['Miscellaneous_Info','Qualification'],axis=1,inplace=True)
df_test.drop(['Miscellaneous_Info','Qualification'],axis=1,inplace=True)

In [491]:
data_train=pd.concat([df_train,df_qualification],axis=1,sort=False)
data_test=pd.concat([df_test,df_qualification_test],axis=1,sort=False)

In [492]:
data_train

,Experience,Rating,Place,Fees,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,24,100,295,100,0,0,0,0,0,1,7,105,92,0,0,0,0,0,0,0
1,12,98,867,350,1,0,0,0,0,0,4,105,12,142,0,0,0,0,0,0
2,9,0,453,300,0,0,0,1,0,0,47,119,141,0,0,0,0,0,0,0
3,12,0,65,250,1,0,0,0,0,0,10,171,14,0,0,0,0,0,0,0
4,20,100,334,250,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5956,19,98,67,300,0,0,0,1,0,0,47,119,60,0,0,0,0,0,0,0
5957,33,0,542,100,0,0,0,0,1,0,47,0,0,0,0,0,0,0,0,0
5958,41,97,216,600,0,0,0,0,1,0,47,0,0,0,0,0,0,0,0,0
5959,15,90,851,100,0,0,0,0,1,0,47,105,78,142,0,0,0,0,0,0


In [493]:
df_x=data_train.drop('Fees',axis=1)
y=data_train['Fees']
df_x.shape,y.shape

((5961, 19), (5961,))

In [494]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x=ss.fit_transform(df_x)
x=pd.DataFrame(x,columns=df_x.columns)
x

,Experience,Rating,Place,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,0.600979,1.220669,-0.469482,-0.389723,-0.553255,-0.458109,-0.372984,-0.494962,2.463975,-1.007883,0.838284,0.643402,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
1,-0.476040,1.178419,1.749064,2.565925,-0.553255,-0.458109,-0.372984,-0.494962,-0.405848,-1.151426,0.838284,-0.767090,1.169337,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
2,-0.745295,-0.891845,0.143333,-0.389723,-0.553255,-0.458109,2.681079,-0.494962,-0.405848,0.906023,1.113437,1.507329,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
3,-0.476040,-0.891845,-1.361555,2.565925,-0.553255,-0.458109,-0.372984,-0.494962,-0.405848,-0.864340,2.135434,-0.731827,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
4,0.241972,1.220669,-0.318218,2.565925,-0.553255,-0.458109,-0.372984,-0.494962,-0.405848,-1.151426,-1.225363,-0.978663,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5956,0.152221,1.178419,-1.353798,-0.389723,-0.553255,-0.458109,2.681079,-0.494962,-0.405848,0.906023,1.113437,0.079206,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
5957,1.408743,-0.891845,0.488526,-0.389723,-0.553255,-0.458109,-0.372984,2.020357,-0.405848,0.906023,-1.225363,-0.978663,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
5958,2.126756,1.157293,-0.775890,-0.389723,-0.553255,-0.458109,-0.372984,2.020357,-0.405848,0.906023,-1.225363,-0.978663,-0.735479,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739
5959,-0.206786,1.009417,1.687007,-0.389723,-0.553255,-0.458109,-0.372984,2.020357,-0.405848,0.906023,0.838284,0.396566,1.169337,-0.622448,-0.441232,-0.379873,-0.309371,-0.267042,-0.224739


In [495]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score

In [496]:
lr=LinearRegression()
sv=SVR()
knn=KNeighborsRegressor()
dt=DecisionTreeRegressor()
rf=RandomForestRegressor()
gb=GradientBoostingRegressor()
xgb=XGBRegressor()
ad=AdaBoostRegressor()

In [497]:
model_list=[]
model_list.append(('LinearRegression',lr))
model_list.append(('SVR',sv))
model_list.append(('Kneighbors',knn))
model_list.append(('DecisionTreeRegressor',dt))
model_list.append(('RandomForestRegressor',rf))
model_list.append(('GradientBoostingRegressor',gb))
model_list.append(('XGBoostRegressor',xgb))
model_list.append(('AdBoostingRegressor',ad))

In [498]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=101,test_size=.20)

In [499]:
Model=[]
R2_score=[]
CS=[]
MSE=[]
MAE=[]
for name,model in model_list:
    print('**********************',name,'************************')
    Model.append(name)
    model.fit(x_train,y_train)
    print(model)
    pred=model.predict(x_test)
    print('\n')
    RS=r2_score(pred,y_test)
    print('r2_score is ',RS)
    R2_score.append(RS*100)
    print('\n')
    cs=cross_val_score(model,x,y,cv=10,scoring='r2').mean()
    print('cross_val is ',cs)
    CS.append(cs*100)
    print('\n')
    mse=mean_squared_error(y_test,pred)
    print('mean square error is',mse)
    MSE.append(mse)
    print('\n')
    mae=mean_absolute_error(y_test,pred)
    print('mean absolute error is',mae)
    MAE.append(mae)
    
    
    

********************** LinearRegression ************************
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)


r2_score is  -4.504485564185943


cross_val is  0.14489589245640738


mean square error is 29678.359768182712


mean absolute error is 138.04230548639538
********************** SVR ************************
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)


r2_score is  -14.817836264412444


cross_val is  0.09088912427714044


mean square error is 31574.30866902427


mean absolute error is 138.67280719254427
********************** Kneighbors ************************
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')


r2_score is  -1.5056006205908243


cross_val is  0.02506687253111961


mean square 

In [500]:
best_model=pd.DataFrame({'Model': Model,'R2_score':R2_score,'CV_Score':CS,'MSE':mse,'MAE': mae})
best_model



,Model,R2_score,CV_Score,MSE,MAE
0,LinearRegression,-450.448556,14.489589,33211.751048,152.364223
1,SVR,-1481.783626,9.088912,33211.751048,152.364223
2,Kneighbors,-150.560062,2.506687,33211.751048,152.364223
3,DecisionTreeRegressor,-39.563279,-60.106120,33211.751048,152.364223
4,RandomForestRegressor,-155.731599,6.919078,33211.751048,152.364223
5,GradientBoostingRegressor,-319.955825,18.588427,33211.751048,152.364223
6,XGBoostRegressor,-318.459766,18.932796,33211.751048,152.364223
7,AdBoostingRegressor,-891.285005,6.906969,33211.751048,152.364223
